In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sep
import pandas as pd
import os
from astropy.table import Table, vstack

In [ ]:
from maddeb.utils import listdir_fullpath, get_data_dir_path

In [ ]:
logging.basicConfig(format="%(message)s", level=logging.INFO)
LOG = logging.getLogger(__name__)

## load the saved results

In [ ]:
density_level = "low"

In [ ]:
data_path = "/sps/lsst/users/bbiswas/MADNESS_results/" + density_level + "_density"
run_name = "kl01" # 12 to 20 galaxies
result_dir_path = os.path.join(data_path, run_name)

In [ ]:
madness_results=[]
scarlet_results=[]
actual_photometry = []
blended_photometry = []

for file_num in range(300):

    madness_results = pd.read_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))

    actual_photometry = pd.read_pickle(os.path.join(data_path, "actual_photometry", str(file_num) + ".pkl"))
    blended_photometry = pd.read_pickle(os.path.join(data_path, "blended_photometry",  str(file_num) + ".pkl"))
    
    madness_results["r_contamination"] = ((blended_photometry["r_phot_flux"].values - actual_photometry["r_phot_flux"].values))/actual_photometry["r_phot_flux"].values

    madness_results.to_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))